# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [142]:
import requests
import time
time.sleep(0.2)
from bs4 import BeautifulSoup
import re
from pymystem3 import Mystem
import datetime
from datetime import datetime, timedelta
import pandas as pd
import json

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [37]:
# определяем список хабов, которые нам интересны
KEYWORDS = ['python', 'парсинг', 'бизнес']

In [3]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [4]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="ru">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="width=1024" name="viewport"/>
  <title>
   Все публикации подряд / Хабр
  </title>
  <meta content="444736788986613" property="fb:app_id">
   <meta content="website" property="og:type"/>
   <meta content="472597926099084" property="fb:pages"/>
   <meta content="Хабр" property="og:site_name"/>
   <link href="https://habr.com/images/habr_ru.png" rel="image_src"/>
   <meta content="https://habr.com/images/habr_ru.png" property="og:image"/>
   <meta content="1200" property="og:image:width"/>
   <meta content="628" property="og:image:height"/>
   <meta content="Все публикации подряд / Хабр" property="og:title"/>
   <meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
   <meta content="71593b225aeafc4e" name="yandex-verification"/>
   <meta cont

In [5]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/naychenko/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/be8/03b/2e2/be803b2e23969442ac082d77b5824bb8.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">naychenko</span>
 </a>
 <span class="post__time">сегодня в 11:42</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/514686/">Бизнес-ландшафт будущего</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/business_models/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Бизнес-модели'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Бизнес-модели</a>, 
    

In [6]:
titles = posts[0].find_all('a', class_ = 'post__title_link')[0].text
titles

'Бизнес-ландшафт будущего'

In [34]:
bodies = soup.find_all('div', class_= 'post__body post__body_crop')
body_0= bodies[0]

In [36]:
body_0.text

'\n\n      В этой статье я хочу поделиться видением развития бизнеса в ближайшие 20 лет, ведь мы находимся в начале трансформации бизнес-ландшафта в новое равновесное состояние.\n\r\nПока эти изменения нечувствительны для многих, но вскоре это коснется абсолютно всех.\n\r\nДанная работа стала результатом анализа большого объема источников, с которыми мне посчастливилось познакомится в рамках работы над проектированием нескольких платформ. В конце статьи я даю ссылки на наиболее важные из этих источников. Так что, эта работа не столько мое творение, сколько обзор и анализ работ других авторов, а также моя трактовка и попытка визуализировать предстоящие трансформации в бизнесе.\n\r\nДля начала, давайте разберемся в бизнес-ландшафте настоящего. Упрощенно, текущий бизнес-ландшафт можно отобразить на следующей схеме:\n\n\n\n(Н) Компании хитов, ориентированные на удовлетворение массовых потребностей клиентов.\n\n(LT) Нишевые компании «Длинного хвоста», ориентированные на удовлетворение персо

In [17]:
link = posts[0].find('a', class_ = 'btn btn_x-large btn_outline_blue post__habracut-btn').attrs.get('href')
link

'https://habr.com/ru/post/514686/#habracut'

In [10]:
string_date = posts[0].find('span', class_= 'post__time').text
string_date

'сегодня в 11:42'

In [12]:
def get_valid_date(string_date):
    today_pattern = r'^сегодня.+'
    yesterday_pattern = r'^вчера.+'
    monthes = {
        'января':'01',
        'февраля':'02',
        'марта':'03',
        'апреля':'04',
        'мая':'05',
        'июня':'06',
        'июля':'07',
        'августа':'08',
        'сентября':'09',
        'октября':'10',
        'ноября':'11',
        'декабря':'12'
    }
    if len(re.findall(today_pattern, string_date)) > 0 :
        today = datetime.now()
        date = today.strftime('%Y-%m-%d')
        
    elif len(re.findall(yesterday_pattern, string_date)) > 0:
        today = datetime.now()
        yesterday =  today - timedelta(days = 1)
        date = yesterday.strftime('%Y-%m-%d')
        
    else:
        string_date = string_date.split(' ')
        date = '-'.join([string_date[2], monthes[string_date[1]], string_date[0]])
        
    return date

In [13]:
test_dates = [
    'сегодня в 9:00',
    '21 июля 2020 в 16:09',
    '21 июля 2020 в 14:53',
    'сегодня в 14:03',
    'вчера в 13:21'
]

In [14]:
for test_date in test_dates:
    print(test_date, 'vs', get_valid_date(test_date))

сегодня в 9:00 vs 2020-08-11
21 июля 2020 в 16:09 vs 2020-07-21
21 июля 2020 в 14:53 vs 2020-07-21
сегодня в 14:03 vs 2020-08-11
вчера в 13:21 vs 2020-08-10


In [30]:
def is_word_found(keywords, fragment):
    pattern = r'(^\s*)(.+)(\s*$)'
    fragment = re.sub(pattern, r'\2', fragment)
    
    fragment = fragment.lower()
    
    m = Mystem()
    lemmas = m.lemmatize(fragment)
    fragment = ' '.join(lemmas)
    
    fragment = re.sub(r'\s{2,3}', r' ', fragment)
    fragment = fragment.strip().split(' ')
    
    flag = False
    for word in fragment:
        if word in keywords:
            flag = True
            break
    
    return flag

In [38]:
is_word_found(KEYWORDS, body_0.text)

True

In [43]:
habr_news = pd.DataFrame()

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    else:
        title = post.find('a', class_ = 'post__title_link').text
        preview = post.find('div', class_= 'post__body post__body_crop').text
        print(title)
        if (is_word_found(KEYWORDS, title)) | (is_word_found(KEYWORDS, preview)):
            date = get_valid_date(post.find('span', class_= 'post__time').text)
            link = post.find('a', class_ = 'btn btn_x-large btn_outline_blue post__habracut-btn')\
            .attrs.get('href')
            row = {'date': date, 'title': title, 'link': link}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

Бизнес-ландшафт будущего
Поймут ли ваши иконки пользователи из других стран? Обзор научных исследований
Политика общего происхождения и CORS: визуальное руководство
Нам нужно поговорить…
Онлайн митап Zabbix об информационной безопасности | 28 августа
Мне кажется, дело не в языке, а в том, как на нем пишут
Эволюция конфигурации .NET
Неофициальный гайд по Active Admin
Riak Cloud Storage. Часть 2. Настройка компонента Riak CS
Язык запросов для TSDB. Улучшаем PromQL (Александр Валялкин, VictoriaMetrics)
Резервное хранение данных. Где, что и как?
Оптимизация. Цикл рассказов «Свойство слоя»
Продуманный front-end. Правильная архитектура для быстрых сайтов
Как прекратить страдать и начать учиться
Продвинутые поиск на NuGet.org
Как правильно подготовить проект печатной платы, чтобы не пришлось его переделывать
Самый мощный сервер Supermicro в Москве на основе AMD Epyc
Как и почему в InnoDB появились индексы на основе В-дерева
Цена tailing'а логов в Kubernetes
Оконные функции своими руками


,date,title,link
0,2020-08-11,Бизнес-ландшафт будущего,https://habr.com/ru/post/514686/#habracut
0,2020-08-11,Неофициальный гайд по Active Admin,https://habr.com/ru/company/domclick/blog/5145...


In [144]:
habr_news.reset_index()

,index,date,title,link,text
0,0,2020-08-11,Онлайн-Круглый стол «Как создать свой игровой ...,https://habr.com/ru/post/514720/#habracut,"\n19 августа (Среда), в 19:00, состоится беспл..."
1,0,2020-08-11,Открытые и персональные данные. Анализ кейса «...,https://habr.com/ru/post/514564/#habracut,"\n\n\r\nДве недели назад, на форумах обнаружил..."
2,0,2020-08-11,Как монетизировать мобильное приложение в 2020...,https://habr.com/ru/post/514710/#habracut,"\nВ этой статье я хочу рассказать о том, как и..."
3,0,2020-08-11,Бизнес-ландшафт будущего,https://habr.com/ru/post/514686/#habracut,\nВ этой статье я хочу поделиться видением раз...
4,0,2020-08-11,"Мне кажется, дело не в языке, а в том, как на ...",https://habr.com/ru/company/skyeng/blog/514674...,\n«Летом между 2 и 3 курсом я пошла искать раб...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [71]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [72]:
posts = soup.find_all('article', class_='post')

In [53]:
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Как прекратить страдать и начать учиться / Блог компании ГК ЛАНИТ / Хабр</title>
<meta content="Успели во время самоизоляции стать суперэффективными и освоить пару новых специальностей? Тоже нет? А ведь все вокруг трубили: «Используй это время с пользой! Кризис – всегда точка роста»...." name="description">
<meta content="ланит, сетевая академия, сетевая академия ланит, обучение, онлайн обучение" name="keywords"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="article" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="https://habr.com/ru/company/lanit/blog/511996/" property="og:url"/>
<meta content="Как прекратить страдать и начать учиться" property="og:title"/>
<meta content="https://habr.com/share/publication/511996/fe0fd04ec707c9b21e42d7d7893a6467/

In [67]:
article = soup.find_all('article', class_='post')

In [68]:
article

[<article class="post post_full" id="post_511996" lang="ru">
 <div class="post__wrapper">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/sheltata/" title="Автор публикации">
 <span class="default-image_mini default-image_green">
 <svg aria-hidden="true" class="icon-svg" height="24" role="img" version="1.1" viewbox="0 0 24 24" width="24"><path d="M21.5 24h-19c-1.379 0-2.5-1.122-2.5-2.5v-19c0-1.379 1.122-2.5 2.5-2.5h19c1.379 0 2.5 1.122 2.5 2.5v19c0 1.379-1.122 2.5-2.5 2.5zm-19-23c-.827 0-1.5.673-1.5 1.5v19c0 .827.673 1.5 1.5 1.5h19c.827 0 1.5-.673 1.5-1.5v-19c0-.827-.673-1.5-1.5-1.5h-19zM15.598 12.385zM19.438 15.417l-.002-.005v-.001c-.875-2.226-2.484-3.054-3.445-3.549l-.273-.143c.029-.497-.025-1.034-.167-1.599l-.128.032.123-.044c-.765-2.152-1.757-2.585-2.632-2.967l-.006-.003-.535-2.121c.357-.065.628-.375.628-.752.001-.423-.342-.765-.765-.765s-.766.342-.766.765c0 .358.248.657.581.74l-.825 1.654-.014-.003-.024-.003c-1.053-.033-1.842.369-

In [69]:
article_text = article[0].find('div', class_= 'post__body post__body_full').text

In [70]:
article_text

'\nУспели во время самоизоляции стать суперэффективными и освоить пару новых специальностей? Тоже нет? А ведь все вокруг трубили: «Используй это время с пользой! Кризис – всегда точка роста». Оказывается, это совершенно нормально, если вы не нашли в себе сил начать осваивать новый язык программирования или вышивание гладью. Психологи говорят, что сильный и продолжительный стресс вредит обучению, а мы все это время прожили в условии постоянно меняющихся правил. Однако хватит «выученной беспомощности», впереди еще много перемен, и суперэффективность нам может пригодиться.\xa0\n\r\nЯ поговорила с преподавателями «Сетевой Академии» (кстати, нашему учебному центру 30 августа исполняется 25 лет!), чтобы узнать, что они думают про обучение дома, и собрала все мысли в этом посте: действительно ли это было время возможностей? Что поможет обучению, если всё вокруг мешает процессу? Чему и где учиться онлайн? Они дали свои рекомендации и рассказали о достойных образовательных платформах.\n\r\nКста

In [77]:
%%time
habr_news = pd.DataFrame()

for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    else:
        title = post.find('a', class_ = 'post__title_link').text
        preview = post.find('div', class_= 'post__body post__body_crop').text
        link = post.find('a', class_ = 'btn btn_x-large btn_outline_blue post__habracut-btn')\
            .attrs.get('href')
        
        article_req = requests.get(link)
        soup = BeautifulSoup(article_req.text, 'html.parser')
        article = soup.find('article', class_='post')
        article_text = article.find('div', class_= 'post__body post__body_full').text
        
        print(title)
        
        print('Проверяю заголовок')
        is_found_flag = is_word_found(KEYWORDS, title)
        
        if is_found_flag:
            print('Слово найдено в заголовке')
        else:
            print('Проверяю превью')
            is_found_flag = is_word_found(KEYWORDS, preview)
            if is_found_flag:
                print('Слово найдено в превью')
            else:
                print('Проверяю текст')
                is_found_flag = is_word_found(KEYWORDS, article_text)
   
        if is_found_flag:
            date = get_valid_date(post.find('span', class_= 'post__time').text)
            row = {'date': date, 'title': title, 'link': link, 'text': article_text }
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])  
habr_news

Онлайн-Круглый стол «Как создать свой игровой бизнес»
Проверяю заголовок
Слово найдено в заголовке
Математические основы кодирования и шифрования
Проверяю заголовок
Проверяю превью
Проверяю текст
Миграция с Zimbra OSE 8.8.15 на Zimbra 9 Open Source от Zextras
Проверяю заголовок
Проверяю превью
Проверяю текст
Почему зарплаты в IT на територии ЕС и Восточной Европы так слабо отличаются?
Проверяю заголовок
Проверяю превью
Проверяю текст
Открытые и персональные данные. Анализ кейса «утечки данных» с Авито
Проверяю заголовок
Проверяю превью
Проверяю текст
В начале 2000-х экономика мира EverQuest по ВНД была между Россией и Болгарией (77-е место)
Проверяю заголовок
Проверяю превью
Проверяю текст
Как монетизировать мобильное приложение в 2020 году?
Проверяю заголовок
Проверяю превью
Проверяю текст
Модульные телефоны — что стало с нашумевшими проектами? Часть 2
Проверяю заголовок
Проверяю превью
Проверяю текст
Делаем игру с управлением улыбкой
Проверяю заголовок
Проверяю превью
Проверяю текст


,date,title,link,text
0,2020-08-11,Онлайн-Круглый стол «Как создать свой игровой ...,https://habr.com/ru/post/514720/#habracut,"\n19 августа (Среда), в 19:00, состоится беспл..."
0,2020-08-11,Открытые и персональные данные. Анализ кейса «...,https://habr.com/ru/post/514564/#habracut,"\n\n\r\nДве недели назад, на форумах обнаружил..."
0,2020-08-11,Как монетизировать мобильное приложение в 2020...,https://habr.com/ru/post/514710/#habracut,"\nВ этой статье я хочу рассказать о том, как и..."
0,2020-08-11,Бизнес-ландшафт будущего,https://habr.com/ru/post/514686/#habracut,\nВ этой статье я хочу поделиться видением раз...
0,2020-08-11,"Мне кажется, дело не в языке, а в том, как на ...",https://habr.com/ru/company/skyeng/blog/514674...,\n«Летом между 2 и 3 курсом я пошла искать раб...


In [145]:
habr_news.reset_index()

,index,date,title,link,text
0,0,2020-08-11,Онлайн-Круглый стол «Как создать свой игровой ...,https://habr.com/ru/post/514720/#habracut,"\n19 августа (Среда), в 19:00, состоится беспл..."
1,0,2020-08-11,Открытые и персональные данные. Анализ кейса «...,https://habr.com/ru/post/514564/#habracut,"\n\n\r\nДве недели назад, на форумах обнаружил..."
2,0,2020-08-11,Как монетизировать мобильное приложение в 2020...,https://habr.com/ru/post/514710/#habracut,"\nВ этой статье я хочу рассказать о том, как и..."
3,0,2020-08-11,Бизнес-ландшафт будущего,https://habr.com/ru/post/514686/#habracut,\nВ этой статье я хочу поделиться видением раз...
4,0,2020-08-11,"Мне кажется, дело не в языке, а в том, как на ...",https://habr.com/ru/company/skyeng/blog/514674...,\n«Летом между 2 и 3 курсом я пошла искать раб...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [140]:
emails = ['boiko_tatjana@mail.ru', 'boiko.andrew.1985@gmail.com', 'boiko_andrew@mail.ru']

In [143]:
avast = pd.DataFrame()
for email in emails:
    req = requests.post(url = 'https://digibody.avast.com/v1/web/leaks',
                    json = {'email':email})
    req_json = json.loads(req.text)
    value = req_json['value']
    for i, item in enumerate(value):
        date_unix = round((value[i]['leak_info']['date'])/1000)
        date = datetime.fromtimestamp(date_unix).strftime("%Y-%m-%d")
        domain = value[i]['domain']
        title = value[i]['leak_info']['title']
        description = value[i]['leak_info']['description']
        row = {'email': email,'date': date, 'domain' : domain, 'title': title, 'description': description}
        avast = pd.concat([avast, pd.DataFrame([row])]) 
avast.reset_index()

,index,email,date,domain,title,description
0,0,boiko_tatjana@mail.ru,2019-06-13,canva.com,Canva,"In May 2019, graphic-design site Canva's datab..."
1,0,boiko.andrew.1985@gmail.com,2017-12-22,,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
2,0,boiko.andrew.1985@gmail.com,2019-02-06,,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
3,0,boiko.andrew.1985@gmail.com,2017-10-09,,Exploit.in combolist,The proliferation of stolen or leaked database...
4,0,boiko.andrew.1985@gmail.com,2019-02-06,,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
5,0,boiko.andrew.1985@gmail.com,2019-01-29,,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
6,0,boiko.andrew.1985@gmail.com,2018-12-21,,Sensitive Source,This source has been marked as sensitive due t...
7,0,boiko.andrew.1985@gmail.com,2016-10-29,vk.com,VK,Popular Russian social networking platform VKo...
8,0,boiko_andrew@mail.ru,2017-12-22,,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
9,0,boiko_andrew@mail.ru,2018-08-24,,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
